In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
path_df = "C:/Users/trela/Python Programming/ICE1 Turn In/02. Exploratory Data Analysis/News_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,File_Name,Content,Category,Complete_Filename,id,News_length
0,001.txt,Autonomous Vehicles 101\r\n\r\nAutonomous vehi...,autonomous vehicles,001.txt-autonomous vehicles,1,3041
1,002.txt,Which Cars Have Self-Driving Features for 2021...,autonomous vehicles,002.txt-autonomous vehicles,1,31287
2,003.txt,Self-driving cars: your complete guide to auto...,autonomous vehicles,003.txt-autonomous vehicles,1,63480
3,004.txt,Self-driving cars will not fix our transportat...,autonomous vehicles,004.txt-autonomous vehicles,1,6257
4,005.txt,How we feel about our cars means the road to a...,autonomous vehicles,005.txt-autonomous vehicles,1,5422


In [4]:
df.loc[1]['Content']

'Which Cars Have Self-Driving Features for 2021?\r\n\r\nNot long ago, fully autonomous vehicles for the masses were said to be just around the corner. As it turns out, making the final leap to full autonomy â€” self-driving on any road at any time â€” remains tantalizingly out of reach both for engineers and safety regulators. And despite all the attention such vehicles have drawn, Americans arenâ€™t exactly clamoring for self-driving cars: The latest AAA annual survey on automated vehicles found that just 14% of drivers would trust riding in a self-driving car, about the same as 2020.\r\n\r\nBut tech features that form the building blocks for self-driving have spread quickly into mainstream vehicles in just a few years (check out our 2016 report here). These systems can now aid the driver with steering, acceleration and braking â€” though we stress that, no matter their capabilities, they are not truly self-driving. All such systems still require the driver to be in charge and ready t

In [5]:
# \r and \n
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [6]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [7]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [8]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [9]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

<ipython-input-9-cdceb98d92a9>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


In [10]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [11]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\trela\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\trela\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [13]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [14]:
df['Content_Parsed_5'] = lemmatized_text_list

In [15]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trela\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [17]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [18]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [19]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

regex = True    ## [ADDED for future-proofing change in default boolean value]
for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-19-0bfa395c13ef>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [20]:
df.loc[5]['Content']

"How Google's Self-Driving Car Will Change Everything\r\n\r\nImagine getting into your car, typingâ€”or, better yet speakingâ€”a location into your vehicleâ€™s interface, then letting it drive you to your destination while you read a book, surf the web, or nap. Self-driving vehiclesâ€”the stuff of science fiction since the first roads were pavedâ€”are coming, and theyâ€™re going to radically change what itâ€™s like to get from point A to point B.\r\n\r\nIn 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google's self-driving project became Waymo.\r\n\r\nSince then, Waymo has invited the public to join the first public trial of autonomous vehicles operated by the Waymo Driver and introduced its first fully autonomous vehicles operated by the Waymo Driver on public roads without anyone in the driverâ€™s seat.\r\n\r\

In [21]:
df.loc[5]['Content_Parsed_1']

"How Google's Self-Driving Car Will Change Everything Imagine getting into your car, typingâ€”or, better yet speakingâ€”a location into your vehicleâ€™s interface, then letting it drive you to your destination while you read a book, surf the web, or nap. Self-driving vehiclesâ€”the stuff of science fiction since the first roads were pavedâ€”are coming, and theyâ€™re going to radically change what itâ€™s like to get from point A to point B. In 2009, Google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. In 2016, Waymo, an autonomous driving technology company, became a subsidiary of Alphabet, and Google's self-driving project became Waymo. Since then, Waymo has invited the public to join the first public trial of autonomous vehicles operated by the Waymo Driver and introduced its first fully autonomous vehicles operated by the Waymo Driver on public roads without anyone in the driverâ€™s seat. KEY TAKEAWAYS  The hype aro

In [22]:
df.loc[5]['Content_Parsed_2']

"how google's self-driving car will change everything imagine getting into your car, typingâ€”or, better yet speakingâ€”a location into your vehicleâ€™s interface, then letting it drive you to your destination while you read a book, surf the web, or nap. self-driving vehiclesâ€”the stuff of science fiction since the first roads were pavedâ€”are coming, and theyâ€™re going to radically change what itâ€™s like to get from point a to point b. in 2009, google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes. in 2016, waymo, an autonomous driving technology company, became a subsidiary of alphabet, and google's self-driving project became waymo. since then, waymo has invited the public to join the first public trial of autonomous vehicles operated by the waymo driver and introduced its first fully autonomous vehicles operated by the waymo driver on public roads without anyone in the driverâ€™s seat. key takeaways  the hype aro

In [23]:
df.loc[5]['Content_Parsed_3']

"how google's self-driving car will change everything imagine getting into your car typingâ€”or better yet speakingâ€”a location into your vehicleâ€™s interface then letting it drive you to your destination while you read a book surf the web or nap self-driving vehiclesâ€”the stuff of science fiction since the first roads were pavedâ€”are coming and theyâ€™re going to radically change what itâ€™s like to get from point a to point b in 2009 google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes in 2016 waymo an autonomous driving technology company became a subsidiary of alphabet and google's self-driving project became waymo since then waymo has invited the public to join the first public trial of autonomous vehicles operated by the waymo driver and introduced its first fully autonomous vehicles operated by the waymo driver on public roads without anyone in the driverâ€™s seat key takeaways  the hype around driverless ca

In [24]:
df.loc[5]['Content_Parsed_4']

"how google self-driving car will change everything imagine getting into your car typingâ€”or better yet speakingâ€”a location into your vehicleâ€™s interface then letting it drive you to your destination while you read a book surf the web or nap self-driving vehiclesâ€”the stuff of science fiction since the first roads were pavedâ€”are coming and theyâ€™re going to radically change what itâ€™s like to get from point a to point b in 2009 google started the self-driving car project with the goal of driving autonomously over ten uninterrupted 100-mile routes in 2016 waymo an autonomous driving technology company became a subsidiary of alphabet and google self-driving project became waymo since then waymo has invited the public to join the first public trial of autonomous vehicles operated by the waymo driver and introduced its first fully autonomous vehicles operated by the waymo driver on public roads without anyone in the driverâ€™s seat key takeaways  the hype around driverless cars h

In [25]:
df.loc[5]['Content_Parsed_5']

"how google self-driving car will change everything imagine get into your car typingâ€”or better yet speakingâ€”a location into your vehicleâ€™s interface then let it drive you to your destination while you read a book surf the web or nap self-driving vehiclesâ€”the stuff of science fiction since the first roads be pavedâ€”are come and theyâ€™re go to radically change what itâ€™s like to get from point a to point b in 2009 google start the self-driving car project with the goal of drive autonomously over ten uninterrupted 100-mile rout in 2016 waymo an autonomous drive technology company become a subsidiary of alphabet and google self-driving project become waymo since then waymo have invite the public to join the first public trial of autonomous vehicles operate by the waymo driver and introduce its first fully autonomous vehicles operate by the waymo driver on public roads without anyone in the driverâ€™s seat key takeaways  the hype around driverless cars have grow rapidly over the 

In [26]:
df.loc[5]['Content_Parsed_6']

" google self-driving car  change everything imagine get   car typingâ€” better yet speakingâ€” location   vehicleâ€™ interface  let  drive    destination   read  book surf  web  nap self-driving vehiclesâ€” stuff  science fiction since  first roads  pavedâ€” come  theyâ€™ go  radically change  itâ€™ like  get  point   point b  2009 google start  self-driving car project   goal  drive autonomously  ten uninterrupted 100-mile rout  2016 waymo  autonomous drive technology company become  subsidiary  alphabet  google self-driving project become waymo since  waymo  invite  public  join  first public trial  autonomous vehicles operate   waymo driver  introduce  first fully autonomous vehicles operate   waymo driver  public roads without anyone   driverâ€™ seat key takeaways   hype around driverless cars  grow rapidly   past several years  many big technology company get behind  concept  google launch  waymo division  develop  market consumer-ready driverless vehicles around  globe   company

In [27]:
df.head(1)

,File_Name,Content,Category,Complete_Filename,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,001.txt,Autonomous Vehicles 101\r\n\r\nAutonomous vehi...,autonomous vehicles,001.txt-autonomous vehicles,1,3041,Autonomous Vehicles 101 Autonomous vehicles â€...,autonomous vehicles 101 autonomous vehicles â€...,autonomous vehicles 101 autonomous vehicles â€...,autonomous vehicles 101 autonomous vehicles â€...,autonomous vehicles 101 autonomous vehicles â€...,autonomous vehicles 101 autonomous vehicles â€...


In [28]:
list_columns = ["File_Name", "Category", "Complete_Filename", "Content", "Content_Parsed_6"]

df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [29]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed
0,001.txt,autonomous vehicles,001.txt-autonomous vehicles,Autonomous Vehicles 101\r\n\r\nAutonomous vehi...,autonomous vehicles 101 autonomous vehicles â€...
1,002.txt,autonomous vehicles,002.txt-autonomous vehicles,Which Cars Have Self-Driving Features for 2021...,cars self-driving feature 2021 long ago fu...
2,003.txt,autonomous vehicles,003.txt-autonomous vehicles,Self-driving cars: your complete guide to auto...,self-driving cars complete guide autonomous ...
3,004.txt,autonomous vehicles,004.txt-autonomous vehicles,Self-driving cars will not fix our transportat...,self-driving cars fix transportation woes i...
4,005.txt,autonomous vehicles,005.txt-autonomous vehicles,How we feel about our cars means the road to a...,feel cars mean road driverless future m...


In [30]:
### "LABELING CODE"

In [31]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

In [32]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [33]:
df.head()

,File_Name,Category,Complete_Filename,Content,Content_Parsed,Category_Code
0,001.txt,autonomous vehicles,001.txt-autonomous vehicles,Autonomous Vehicles 101\r\n\r\nAutonomous vehi...,autonomous vehicles 101 autonomous vehicles â€...,autonomous vehicles
1,002.txt,autonomous vehicles,002.txt-autonomous vehicles,Which Cars Have Self-Driving Features for 2021...,cars self-driving feature 2021 long ago fu...,autonomous vehicles
2,003.txt,autonomous vehicles,003.txt-autonomous vehicles,Self-driving cars: your complete guide to auto...,self-driving cars complete guide autonomous ...,autonomous vehicles
3,004.txt,autonomous vehicles,004.txt-autonomous vehicles,Self-driving cars will not fix our transportat...,self-driving cars fix transportation woes i...,autonomous vehicles
4,005.txt,autonomous vehicles,005.txt-autonomous vehicles,How we feel about our cars means the road to a...,feel cars mean road driverless future m...,autonomous vehicles


In [34]:
### SPLIT DATA OR TESTING AND TRAINING

In [35]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [36]:
### TEXT REPRESENTATION

In [37]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [38]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1898, 300)
(335, 300)


In [39]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. profit
. economy
. oil
. growth
. bank
  . Most correlated bigrams:
. last year
. year old

# 'entertainment' category:
  . Most correlated unigrams:
. music
. best
. award
. star
. film
  . Most correlated bigrams:
. mr blair
. prime minister

# 'politics' category:
  . Most correlated unigrams:
. minister
. blair
. party
. election
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. win
. club
. side
. team
. match
  . Most correlated bigrams:
. say mr
. year old

# 'tech' category:
  . Most correlated unigrams:
. digital
. technology
. computer
. software
. users
  . Most correlated bigrams:
. year old
. say mr



In [40]:
bigrams

['tell bbc', 'last year', 'prime minister', 'mr blair', 'year old', 'say mr']

In [41]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)